In [256]:
import snappy
from sage.all import Rational, QQ, matrix, sgn, QuadraticForm


def path_to_amphicheiral(mfld):
    # TODO try to find an amphicheiral parent and filling
    return []


#def hodgson_kerckhof_slopes(mfld,slopes):
#    snappy.verify.cusp_translations_for_manifold(mfld, True, areas=[1]*mfld.num_cusps())


def get_fillings(mfld):
    """
    Returns the integer fillings for each cusp of the given manifold
    """
    return [(int(c.filling[0]), int(c.filling[1])) for c in mfld.cusp_info()]
    

def eta_invariant(mfld, debug = False):
    """
    Attempts to compute the eta invariant of the given manifold.
    """
    # path = path_to_amphicheiral(mfld)
    # if len(path) == 0:
    #    raise Exception("Failed to find amphicheiral parent.")
    fillings = get_fillings(mfld)
    if (0,0) in fillings:
        raise Exception("Only closed manifolds are supported")
    fu_and_lengths = snappy.verify.verified_complex_volume_torsion(mfld, normalize = False)
    dedekind_sum = 4 * sum(dedekind(q, p) for p, q in fillings)
    slope_correction = sum((r(p, q) - q)/ (3 * p) for p, q in fillings)
    signature = signature_Y(mfld)
    eta = fu_and_lengths.imag()/(3 * sage.all.pi**2) + dedekind_sum + slope_correction - signature
    if debug:
        print("Vol/pi^2:", sage.all.N(fu_and_lengths/sage.all.pi**2))
        print("4 Dedekind sum:", dedekind_sum)
        print("Slope correction:", slope_correction)
        print("Signature:", signature)
        cs = (snappy.verify.verified_complex_volume_torsion(M)/(2*sage.all.pi**2)).imag()
        print("CS:", sage.all.N(cs))
        print("3 eta - 2 CS:", sage.all.N(3 * eta - 2 * cs))
    return eta

def eta_change_under_filling(parent, slopes):
    pass


def dedekind(a, c):
    """
    Returns the Dedekind function of a and c
    """
    half = Rational('1/2')
    def fdmh(x):
        return x - x.floor() - half
    return sgn(c) * sum(fdmh(Rational(k)/c) * fdmh(a * Rational(k)/c) for k in range(1, abs(c)))


def sr(p, q):
    """
    Return integers s and r such that p s - q r = gcd(p,q) and 0 <= r/p < 1
    """
    _, s, mr = sage.arith.misc.XGCD(p, q)
    r = -mr
    if r/p < 0 or r/p > 1:
        x = (r/p).floor()
        r = r - x * p
        s = s - x * q
    return s, r

                           
def r(p,q):
    """
    Return integer r such that 0 <= r/p < 1 and there is an integer s with p s - q r = gcd(p,q)
    """
    return sr(p,q)[1]
                           

def cusp_homology_kernel(mfld):
    G = mfld.fundamental_group()
    f = snappy.snap.nsagetools.MapToFreeAbelianization(G)
    ml_pairs = G.peripheral_curves()
    images = [f(pair[0]) for pair in ml_pairs] + [f(pair[1]) for pair in ml_pairs]
    kernel =  matrix(ZZ, images).left_kernel()
    return kernel


def signature_Y(mfld, debug = False):
    fillings = get_fillings(mfld)
    num_cusps = len(fillings)
    if (0,0) in fillings:
        raise Exception("Manifold not closed")
    parent = mfld.copy()
    parent.dehn_fill([(0,0)] * num_cusps)
    kernel = cusp_homology_kernel(parent)
    longitude_mat = matrix(QQ, num_cusps).stack(identity_matrix(num_cusps))
    filling_mat = matrix(QQ, 2 * num_cusps, num_cusps)
    slope_mat = matrix(QQ, num_cusps)
    for i, slope in enumerate(fillings):
        filling_mat[i, i] = slope[0]
        filling_mat[i + num_cusps, i] = slope[1]
        if slope[0] != 0:
            slope_mat[i, i] = slope[1]/slope[0]
    
    image = longitude_mat.augment(filling_mat).column_space()
    intersection_mat = image.intersection(kernel).basis_matrix()
    meridian_proj = intersection_mat[:, :num_cusps]
    longitude_proj = intersection_mat[:, num_cusps:]

    qf = QuadraticForm(meridian_proj *
        (longitude_proj - meridian_proj * slope_mat).transpose())
    if debug:
        print(kernel)
        print(longitude_mat)
        print(filling_mat)
        print(image)
        print(intersection_mat)
        print(meridian_proj)
        print(longitude_proj)
        print(slope_mat)
        print(qf)
    return qf.signature()

In [254]:
snap_data = {'m004(2,3)': {
'eta': -0.5539179101259940238368343194,
'cs12': 0.1691231348110089642447485209,
'cs': -0.3308768651889910357552514791,
'3eta-2cs': -1.000000000000000000000000000
},
'm004(-2,7)':{ 
'eta': 1.190321994685096423781893464,
'cs12': -0.2145170079723553643271598033,
'cs': 0.2854829920276446356728401967,
'3eta-2cs': 3.000000000000000000000000000
},
'm004(3,-17)':{
'eta': 2.117630266895371041664134684,
'cs12': 0.1764454003430565624962020260,
'cs': 0.1764454003430565624962020260,
'3eta-2cs': 6.000000000000000000000000000
}}

In [255]:
for name in ('m004(2,3)', 'm004(-2,7)', 'm004(3,-17)'):
    M = snappy.Manifold(name)
    eta = eta_invariant(M, debug = True)
    snap_eta = snap_data[name]['eta']
    snap_cs = snap_data[name]['cs']
    snap_cs12 = snap_data[name]['cs12']
    print(name, ':', sage.all.N(eta))
    print('snap eta:', sage.all.N(snap_eta))
    print('3 our eta - 2 snap cs mod 1:', sage.all.N(3*eta - 2*snap_cs))
    print('3 our eta - 2 snap cs mod 1/2:',sage.all.N(3*eta - 2*snap_cs12))
    print('3 snap eta - 2 snap cs mod 1:', sage.all.N(3*snap_eta - 2*snap_cs))
    print('3 snap eta - 2 snap cs mod 1/2:', sage.all.N(3*snap_eta - 2*snap_cs12))

Vol/pi^2: 0.0880037238607373 - 0.580876865188991*I
4 Dedekind sum: 0
Slope correction: -1/3
Signature: 0
CS: -0.0808768651889912
3 eta - 2 CS: -1.41912313481101
m004(2,3) : -0.526958955062997
snap eta: -0.553917910125994
3 our eta - 2 snap cs mod 1: -0.919123134811009
3 our eta - 2 snap cs mod 1/2: -1.91912313481101
3 snap eta - 2 snap cs mod 1: -1.00000000000000
3 snap eta - 2 snap cs mod 1/2: -2.00000000000000
Vol/pi^2: 0.0999393994297812 + 1.03548299202764*I
4 Dedekind sum: 0
Slope correction: 4/3
Signature: 0
CS: 0.0354829920276446
3 eta - 2 CS: 4.96451700797236
m004(-2,7) : 1.67849433067588
snap eta: 1.19032199468510
3 our eta - 2 snap cs mod 1: 4.46451700797236
3 our eta - 2 snap cs mod 1/2: 5.46451700797236
3 snap eta - 2 snap cs mod 1: 3.00000000000000
3 snap eta - 2 snap cs mod 1/2: 4.00000000000000
Vol/pi^2: 0.102337858863250 + 2.92644540034306*I
4 Dedekind sum: 2/9
Slope correction: 19/9
Signature: 0
CS: -0.0735545996569402
3 eta - 2 CS: 10.0735545996569
m004(3,-17) : 3.3088

In [249]:
def eta_cusped_estimate(mfld, p, q):
    M1 = mfld.copy()
    M2 = mfld.copy()
    M1.dehn_fill((p,q))
    M2.dehn_fill((p,-q))
    return (eta_invariant(M1) + signature_Y(M1) + eta_invariant(M2) + signature_Y(M2))/2

sage.all.N(eta_cusped_estimate(snappy.Manifold('m004'), -3**15, 2**10))

148.333333239530

In [248]:
-3**10, 2**10

(-59049, 1024)

In [209]:
M = snappy.Manifold('m004')
print(M.chern_simons())
M.dehn_fill(((3,-17)))
print(M)
print(M.solution_type())
print(sage.all.N(M.complex_volume()/(2 * sage.all.pi**2)))
print(sage.all.N(snappy.verify.verified_complex_volume_torsion(M)/(2 * sage.all.pi**2)))
print(sage.all.N((M.complex_volume()-snappy.verify.verified_complex_volume_torsion(M).center())/sage.all.pi**2).imag())
print(sage.all.N(snappy.verify.verified_complex_volume_torsion(M)/(2*sage.all.pi**2)).imag())
print(M.chern_simons())
print(M.cusp_info()[0]['complete?'])

9.00000000000000e-17
m004(3,-17)
all tetrahedra positively oriented
0.102337858863250 + 0.176445400343057*I
0.102337858863250 - 0.0735545996569456*I
0.500000000000004
-0.0735545996569456
0.176445400343057
False


In [203]:
-0.294218398627782-0.176445400343057

-0.470663798970839

In [155]:
sage.all.N((4.93480220054467*2)/(sage.all.pi**2))

0.999999999999998

In [156]:
0.3345758331290873920187961496-0.5

-0.165424166870913

In [157]:
e = eta_invariant(M, debug = True)
sage.all.N(e)

Vol/pi^2: 0.202142151164706 + 0.354260019012955*I
4 Dedekind sum: -3/2
Slope correction: 1/6
Signature: 0


-1.21524666032902

In [137]:
dedekind(7,-20)

5/8

In [152]:
sage.all.N(6*e - 6 * (-1.381913326995681665208110748))

0.999999999999999

In [79]:
a = snappy.verify.verified_complex_volume_torsion(M, normalize = True)

In [81]:
a.center()

1.73198277872576 + 1.66946002975328*I

In [68]:
sage.all.N(1.6694600298/(2*sage.all.pi))

0.265702816037013

In [426]:
m, l = snappy.snap.peripheral.peripheral_cohomology_basis(snappy.Manifold('m006'))
m.dual_cellulation.integral_cohomology_basis()

In [404]:
nonzero_signature = [N for N in snappy.OrientableClosedCensus if signature_Y(N) != 0]

In [405]:
len(nonzero_signature)

10428

In [441]:
sr(-7,4)

1 -2


(1, -2)

In [440]:
(3)*7-(5)*4

1

In [406]:
nonzero_signature[:30]

[m003(-3,1),
 m003(-2,3),
 m007(3,1),
 m003(-4,3),
 m009(4,1),
 m003(-3,4),
 m003(-4,1),
 m003(-5,3),
 m007(1,2),
 m007(4,1),
 m007(3,2),
 m006(3,1),
 m003(-5,4),
 m006(-3,2),
 m015(5,1),
 m007(-3,2),
 m009(5,1),
 m010(-2,3),
 m009(-5,1),
 m011(1,3),
 m006(1,3),
 m007(-5,1),
 m009(1,2),
 m016(-3,2),
 m007(5,1),
 m006(-1,3),
 m017(-3,2),
 m006(3,2),
 m010(-1,3),
 m011(2,3)]

In [407]:
nonzero_signature_pairs = [(N, signature_Y(N)) for N in nonzero_signature]

In [409]:
nonzero_signature_pairs[:300]

[(m003(-3,1), -1),
 (m003(-2,3), -1),
 (m007(3,1), -1),
 (m003(-4,3), -1),
 (m009(4,1), 1),
 (m003(-3,4), -1),
 (m003(-4,1), -1),
 (m003(-5,3), -1),
 (m007(1,2), -1),
 (m007(4,1), -1),
 (m007(3,2), -1),
 (m006(3,1), -1),
 (m003(-5,4), -1),
 (m006(-3,2), 1),
 (m015(5,1), -1),
 (m007(-3,2), 1),
 (m009(5,1), 1),
 (m010(-2,3), 1),
 (m009(-5,1), 1),
 (m011(1,3), -1),
 (m006(1,3), -1),
 (m007(-5,1), -1),
 (m009(1,2), -1),
 (m016(-3,2), 1),
 (m007(5,1), -1),
 (m006(-1,3), 1),
 (m017(-3,2), 1),
 (m006(3,2), -1),
 (m010(-1,3), 1),
 (m011(2,3), -1),
 (m006(4,1), -1),
 (m006(-5,1), -1),
 (m009(3,2), 1),
 (m006(-2,3), 1),
 (m006(2,3), -1),
 (m017(-1,3), 1),
 (m023(-4,1), -1),
 (m007(5,2), -1),
 (m006(-5,2), 1),
 (m036(-3,2), 1),
 (m015(6,1), -1),
 (m016(-2,3), 1),
 (m010(-4,3), 1),
 (m019(3,2), -1),
 (m011(-3,1), 1),
 (m010(4,1), -1),
 (m007(-6,1), -1),
 (m010(3,2), -1),
 (m016(-4,1), 1),
 (m009(6,1), 1),
 (m009(-6,1), 1),
 (m007(-5,2), 1),
 (m015(-5,1), -1),
 (m010(1,3), -1),
 (m016(3,2), -1),
 (

In [414]:
for X in snappy.OrientableCuspedCensus:
    if X.num_cusps() == 2:
        X.dehn_fill([(3,1),(11,-2)])
        if X.solution_type() == 'all tetrahedra positively oriented':
            sig = signature_Y(X)
            if sig != 0:
                twocusp_nonzero.append((X,sig))

In [415]:
twocusp_nonzero

[(s959(-2,7)(3,11), -1),
 (v2444(-2,7)(3,11), -1),
 (v2644(-2,7)(3,11), 1),
 (v3021(-2,7)(3,11), 1),
 (v3127(-2,7)(3,11), 1),
 (v3292(-2,7)(3,11), -1),
 (v3360(-2,7)(3,11), -1),
 (v3426(-2,7)(3,11), 1),
 (v3546(-2,7)(3,11), -1),
 (v3548(-2,7)(3,11), -1),
 (t06250(-2,7)(3,11), 2),
 (t07890(-2,7)(3,11), 2),
 (t08207(-2,7)(3,11), -2),
 (t08869(-2,7)(3,11), -1),
 (t09857(-2,7)(3,11), -1),
 (t10520(-2,7)(3,11), 1),
 (t10700(-2,7)(3,11), -1),
 (t10875(-2,7)(3,11), -2),
 (t11698(-2,7)(3,11), -1),
 (t11708(-2,7)(3,11), 1),
 (t11709(-2,7)(3,11), -1),
 (t11714(-2,7)(3,11), -1),
 (t11761(-2,7)(3,11), -1),
 (t11792(-2,7)(3,11), -1),
 (t11817(-2,7)(3,11), -1),
 (t12032(-2,7)(3,11), 1),
 (t12037(-2,7)(3,11), -1),
 (t12044(-2,7)(3,11), -1),
 (t12049(-2,7)(3,11), -1),
 (t12052(-2,7)(3,11), -1),
 (t12054(-2,7)(3,11), -1),
 (t12190(-2,7)(3,11), -1),
 (t12256(-2,7)(3,11), -1),
 (t12348(-2,7)(3,11), -1),
 (t12486(-2,7)(3,11), 1),
 (t12501(-2,7)(3,11), -1),
 (t12543(-2,7)(3,11), 1),
 (t12598(-2,7)(3,11), -

In [369]:
a = matrix(QQ, [[-3/2,-3/2],[-3/2,3/2],[1/2,-1/2],[1/2,1/2]])
a = matrix(QQ, [[-3/2,-3/2],[-3/2,3/2],[1/2,-1/2],[1/2,1/2]])
print(a)
a.column_space()

[-3/2 -3/2]
[-3/2  3/2]
[ 1/2 -1/2]
[ 1/2  1/2]


Vector space of degree 4 and dimension 2 over Rational Field
Basis matrix:
[   1    0    0 -1/3]
[   0    1 -1/3    0]

In [370]:
b = matrix(QQ, [0,0;0,0;1,0;0,1])

SyntaxError: invalid syntax (3458940913.py, line 1)

In [234]:
a.intersection(b)

Free module of degree 4 and rank 2 over Integer Ring
Echelon basis matrix:
[ 2  4 -2 -1]
[ 0 12 -6  0]

In [145]:
M = snappy.OrientableClosedCensus[231]

In [318]:
N = snappy.Manifold('m203')
G = N.fundamental_group()
f = snappy.snap.nsagetools.MapToFreeAbelianization(G)
ml_pairs = G.peripheral_curves()
images = [f(pair[0]) for pair in ml_pairs] + [f(pair[1]) for pair in ml_pairs]
kernel =  matrix(ZZ, images).left_kernel()

In [322]:
f(ml_pairs[0][0])

(1, 2)

In [324]:
f(ml_pairs[1][1])

(-3, -6)

In [142]:
half = snappy.number.Number('5/4')

In [15]:
type(half)

<class 'snappy.number.Number'>

In [16]:
for L in snappy.LinkExteriors:
    if L.num_cusps() > 1 and L.solution_type() == 'all tetrahedra positively oriented' :
        if L.homological_longitude(cusp = 0) is None:
            print(L)
            break

6^2_2(0,0)(0,0)


In [17]:
M = snappy.Manifold('6^2_2')
M.identify()

[m203(0,0)(0,0), 6^2_2(0,0)(0,0), L6a2(0,0)(0,0), otet04_00001(0,0)(0,0)]

In [18]:
M.solution_type()

'all tetrahedra positively oriented'

In [19]:
L.DT_code()

[(8, 10, 12), (4, 2, 6)]

In [42]:
kernel

Free module of degree 2 and rank 1 over Integer Ring
Echelon basis matrix:
[0 1]

In [23]:
M.cusp_translations()

[(-3.37367044429307e-16 + 1.31607401295249*I, 2.27950705695478),
 (-1.68683522214653e-16 + 1.31607401295249*I, 2.27950705695478)]

In [40]:
matrix(ZZ, images).left_kernel()

Free module of degree 2 and rank 1 over Integer Ring
Echelon basis matrix:
[0 1]

In [26]:
M = snappy.LinkExteriors[231]
M.identify()

[10_148(0,0), K10n12(0,0)]

In [33]:
snappy.verify.cusp_translations_for_manifold(M, True, areas = [1])

[(0.5372849659118? + 0.9306048591021?*I, 1.0745699318236?)]

In [10]:
S = M.symmetry_group()

In [13]:
S.is_amphicheiral?

Docstring:     
SymmetryGroup.is_amphicheiral(self)

   Return whether the manifold has an orientation reversing symmetry.

   >>> S = Manifold('m004').symmetry_group()
   >>> S.is_amphicheiral()
   True
Init docstring: Initialize self.  See help(type(self)) for accurate signature.
File:           
Type:           builtin_function_or_method


In [14]:
raise Exception("Noooo")

Exception: Noooo